# Simpsons characters dialogs

In [63]:
import pandas as pd
from flask import Flask, request

In [64]:
df = pd.read_csv('input/simpsons_dataset.csv', engine = 'python')
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [65]:
nombres = { 
    'raw_character_text':'Characters',
    'spoken_works':'Text'
}
df = df.rename(columns=nombres)

# Create characters groups

In [66]:
df = df = df.dropna()
df = df[(df['Characters'].isin(('Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abraham Simpson', 'Milhouse Van Houten', 'Ralph Wiggum', 'Nelson Muntz', 'Seymour Skinner', 'Edna Krabappel', 'Groundskeeper Willie', 'Jimbo Jones', 'Martin Prince', 'Moe Szyslak', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson')))]
df.head()

,Characters,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
5,Martin Prince,I don't think there's anything left to say.
7,Bart Simpson,Victory party under the slide!
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!


In [67]:
df['Groups'] = df['Groups'][df['Characters'].isin(('Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abraham Simpson'))] = 'Home'
df['Groups'][df['Characters'].isin(('Milhouse Van Houten', 'Ralph Wiggum', 'Nelson Muntz', 'Seymour Skinner', 'Edna Krabappel', 'Groundskeeper Willie', 'Jimbo Jones', 'Martin Prince'))] = 'School'
df['Groups'][df['Characters'].isin(('Moe Szyslak', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson'))] = 'Bar'

In [68]:
df_reduced = df.head(100)
df_reduced.head()

,Characters,spoken_words,Groups
1,Lisa Simpson,Where's Mr. Bergstrom?,Home
3,Lisa Simpson,That life is worth living.,Home
5,Martin Prince,I don't think there's anything left to say.,School
7,Bart Simpson,Victory party under the slide!,Home
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!,Home


# Including Simpsons characters on MongoDB

In [69]:
import requests

In [70]:
for character in pd.unique(df_reduced['Characters']):
    data = requests.get(f"http://localhost:8800/user/create/{character}").content
print(data)

b'\n    <img src="https://cdn.memegenerator.es/imagenes/memes/full/4/19/4199711.jpg">\n    '


# Creating chats

In [71]:
for group in pd.unique(df_reduced['Groups']):
    data = requests.get(f"http://localhost:8800/chat/create/{group}").content
print(data)

b'Chat already exists'


# Adding characters to chat

In [72]:
#Devuelve TypeError got an unexpected keyword argument
for character in pd.unique(df_reduced['Characters']):
    print(character)
    usersdata = requests.get(f"http://localhost:8800/chat/adduser/{character}").text
    print(usersdata)

Lisa Simpson
{
  "error": "name 'df_reduced' is not defined"
}

Martin Prince
{
  "error": "name 'df_reduced' is not defined"
}

Bart Simpson
{
  "error": "name 'df_reduced' is not defined"
}

Nelson Muntz
{
  "error": "name 'df_reduced' is not defined"
}

Milhouse Van Houten
{
  "error": "name 'df_reduced' is not defined"
}

Homer Simpson
{
  "error": "name 'df_reduced' is not defined"
}

Marge Simpson
{
  "error": "name 'df_reduced' is not defined"
}

Moe Szyslak
{
  "error": "name 'df_reduced' is not defined"
}



In [60]:
for group in pd.unique(df_reduced['Groups']):
    data = requests.get(f"http://localhost:8800/chat/adduser/{character}").content
print(data)

b'{\n  "error": "name \'df_reduced\' is not defined"\n}\n'


# Adding messages to chat

In [48]:
'''
Characters from the same group will be on the same MongoDB document conversation
'''
for index, row in df_reduced.iterrows():  
    data = requests.get(f"http://localhost:8800/chat/{row['Groups']}/{row['Characters']}/{row['spoken_words']}").content
print(data)

b'Great. Included'


# Pruebas

In [30]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps

In [31]:
client = MongoClient("mongodb://localhost/Conversations")
db = client.get_database()


In [95]:
for index, row in df_reduced.iterrows(): 
    query = list(db['Conversations'].find({'Group':row['Groups']}))[0]['Message']
    print(query)

[{'username': 'Miss Hoover', 'message': "No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it."}, {'username': 'Lisa Simpson', 'message': "Where's Mr. Bergstrom"}, {'username': 'Miss Hoover', 'message': "I don't know. Although I'd sure like to talk to him. He didn't touch my lesson plan. What did he teach you"}, {'username': 'Lisa Simpson', 'message': 'That life is worth living.'}, {'username': 'Edna Krabappel-Flanders', 'message': "The polls will be open from now until the end of recess. Now, just in case any of you have decided to put any thought into this, we'll have our final statements. Martin"}, {'username': 'Edna Krabappel-Flanders', 'message': 'Bart'}, {'username': 'Bart Simpson', 'message': 'Victory party under the slide!'}, {'username': 'nan', 'message': 'nan'}, {'username': 'Lisa Simpson', 'message': 'Mr. Bergstrom! Mr. Bergstrom!'}, {'username': 'Landlady', 'message': 'He

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
def addUser(username):
    #Look for user's group:
    grupo = list(df_reduced['Groups'][df_reduced['Characters'] == username])[0]
    #Look for user's id
    i = list(db['Users'].find({'username': username}, {'_id':1}))[0]['_id']
    #Look for group's id
    ig = list(db['Conversations'].find({'Group':grupo}, {'_id':1}))[0]['_id']
    query = list(db['Conversations'].update({"_id":ig}, {'$push': {'Characters': {'username': username, '_id':i}}}))
    return 'User added to the chat'

In [47]:
addUser('Moe Szyslak')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


'User added to the chat'

In [46]:
addUser('Lisa Simpson')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


'User added to the chat'